In [2]:
import psycopg2
import pandas as pd
from ftplib import FTP_TLS
import ftplib
import ssl
import re

class SmartFTP(FTP_TLS):
        def makepasv(self):
            invalidhost, port = super(SmartFTP, self).makepasv()
            return self.host, port

        def ntransfercmd(self, cmd, rest=None):
            conn, size = ftplib.FTP.ntransfercmd(self, cmd, rest)
            if self._prot_p:
                conn = self.context.wrap_socket(conn,
                                                server_hostname=self.host,
                                                session=self.sock.session)  # this is the fix
            return conn, size

mapping = {
        "3OB05285": "12P52852",
        "3OB05287": "14P52874",
        "3OB05259": "15P52595",
        "3OB01283": "16P52836",
        "3OB05286": "17P52867",
        "3OB05294": "18P529422",
        "3OB05295": "19P529523",
        "3OB05255": "1P525501",
        "3OB05292": "20P529221",
        "3OB05303": "21P530324",
        "3OB05306": "22P530624",
        "3OB05305": "23P530525",
        "3OB05311": "24P531126",
        "3OB05314": "25P531426",
        "3OB05315": "26P531527",
        "3OB05322": "27P532248",
        "3OB05323": "28P532349",
        "3OB05324": "29P532451",
        "3OB05258": "2P525802",
        "3OB05326": "30P532652",
        "3OB05327": "31P532753",
        "3OB05345": "32P534563",
        "3OB05346": "33P534612",
        "3OB05347": "34P534728",
        "3OB05360": "35P536046",
        "3OB05361": "36P536158",
        "3OB05362": "37P536299",
        "3OB05363": "38P536381",
        "3OB05364": "39P536458",
        "3OB05284": "3P528403",
        "3OB05365": "40P536571",
        "3OB05366": "41P536622",
        "3OB05367": "42P536787",
        "3OB05256": "4P525604",
        "3OB05257": "5P525705",
        "3OB05288": "6P528806",
        "3OB05260": "7P526007",
        "3OB05289": "8P528918",
        "3OB05261": "9P526109",
        "3OB05254": "SIPM5254"
    }
#index_symbols = {'BRAG', 'AFRM', 'META', 'MSFT', 'ZIM', 'VWM', 'TSLA', 'HDB', 'NVDA', 'AMC', 'SPY'}


def get_activity(date):
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
            host="10.7.8.59",
            database="fixtransactions",
            user="scalp",
            password="QAtr@de442",
            port="5432"
    )

    query = f"""
            SELECT trade_timestamp, cl_order_id, clearing_account, symbol, quantity, price, broker_id, trade_type_id, exec_id, trading_date, liquidity_flag as TAG9882, liquidity_flag_broker as TAG7000, last_mkt, target_comp_id, is_spread, contra_broker, contra_trader, contra_capacity, contra_cmta
            FROM trades
            WHERE trading_date = '{date}'
            and clearing_account like '3OB%'
            """
    # Open a cursor
    cursor = conn.cursor()

    # Execute the query
    cursor.execute(query)

    # Fetch all the results
    results = cursor.fetchall()

    # Close the cursor
    cursor.close()

    df = pd.DataFrame(results, columns = ['Date & Time', 'OrderID', 'Account Num', 'Contract/Symbol', 'Quantity', 'Price', 'Broker', 'trade_type_id', 'ExecID', 'trading_date', 'TAG9882', 'TAG7000', 'Exchange', 'target_comp_id', 'is_spread', 'contra_broker', 'contra_trader', 'contra_capacity', 'contra_cmta'])
    df.drop(columns=["Broker", "trade_type_id"], inplace=True)
    df.replace({'INCADROP': 'INet', 'EHUB' : 'Dash', 'VESDROP' : 'Volant'}, inplace=True)
    df.rename(columns={'target_comp_id':'Broker'}, inplace=True)
    df["Account Num"] = df["Account Num"].replace(mapping)
    df[["Symbol", "Expiration", "Call/Put", "Strike"]] = df["Contract/Symbol"].apply(lambda x: extract_data(x))
    df['Option/Equity'] = df['Contract/Symbol'].apply(is_option_or_equity)
    #df.drop(columns=['Contract/Symbol'], inplace=True)
    df["Side"] = df["Quantity"].apply(lambda x: "Buy" if x > 0 else "Sell")
    def set_liq_code(row):
        if row['Broker'] == 'Dash' and row['Exchange'] == 'C2OX':
            if pd.isna(row['TAG9882']) or row['TAG9882'] == '':
                print(f"Debug: Broker=Dash, Exchange=C2OX, but TAG9882 is empty. Row: {row}")
            return row['TAG9882']
        elif row['Exchange'] in ['XCBO', 'EDGO', 'C2OX', 'BATO']:
            if pd.isna(row['TAG7000']) or row['TAG7000'] == '':
                print(f"Debug: Exchange in ['XCBO', 'EDGO', 'C2OX', 'BATO'], but TAG7000 is empty. Row: {row}")
            return row['TAG7000']
        else:
            if pd.isna(row['TAG9882']) or row['TAG9882'] == '':
                print(f"Debug: No conditions met, but TAG9882 is empty. Row: {row}")
            return row['TAG9882']
    df['liq_code'] = df.apply(set_liq_code, axis=1)        
    return df 
def is_option_or_equity(symbol):
    # Regular expression pattern to match option symbols
    option_pattern = r'[A-Z]+\d{8}[CP]\d+(\.\d+)?'
    
    if re.fullmatch(option_pattern, symbol):
        return 'Option'
    else:
        return 'Equity'
def activity_run(date):
    activity_df = get_activity(date)
    get_file(date)
    penny_dict = format_file(date)

    # Assuming dict is your dictionary and df is your DataFrame
    activity_df['Penny'] = activity_df['Symbol'].isin(penny_dict.values())
    activity_df['Penny'] = activity_df['Penny'].replace({True: 'penny', False: 'non-penny'})
    activity_df['Spread/Single'] = activity_df['is_spread'].replace({True: 'Spread', False : 'Single'})
    activity_df.drop(columns='is_spread', inplace=True)
    return activity_df
def extract_data(row):
    if row.endswith("0"):
        symbol = ""
        expiration = ""
        call_put = ""
        Strike = ""

        # Extract symbol
        for char in row:
            if char.isalpha() and char.isupper():
                symbol += char
            else:
                break

        # Extract expiration
        expiration = row[len(symbol):len(symbol)+8]

        # Extract call/put
        call_put = row[len(symbol)+8]

        # Extract strike price
        Strike = row[len(symbol)+9:]

        # Validate data
        if len(symbol) == 0 or len(expiration) != 8 or call_put not in ["C", "P"] or not Strike.replace('.', '', 1).isdigit():
            return pd.Series([None, None, None, None],
                            index=["Symbol", "Expiration", "Call/Put", "Strike"])
        else:
            return pd.Series([symbol, expiration, call_put, Strike],
                            index=["Symbol", "Expiration", "Call/Put", "Strike"])
    else:
        return pd.Series([None, None, None, None],
                        index=["Symbol", "Expiration", "Call/Put", "Strike"])
def download_from_ftp(ftp, filename, date):
    locations = f'/home/elliott/Development/scripts/jupyter_notebooks/penny_pilot_{date}.txt'
    ftp.retrbinary("RETR " + filename, open(locations,"wb").write, 1024)
def get_file(date):  
    filelist=[]      
    ftp = SmartFTP()
    ftp.ssl_version = ssl.PROTOCOL_SSLv23
    ftp.connect('ftp.tradingblock.com', 21, 100)
    ftp.auth()
    ftp.prot_p()
    ftp.login(user='tbftpuser02', passwd='Tr@d1ngBfTp85296')
    ftp.cwd("/")
    ftp.dir('-t',filelist.append)
    file = f'penny_pilot/penny_pilot_{date}.txt'
    download_from_ftp(ftp, file, date)     
    ftp.quit()       
def format_file(date):
    df = pd.read_csv(f'/home/elliott/Development/scripts/jupyter_notebooks/penny_pilot_{date}.txt', delimiter='\t')  # assuming it's comma-separated
    # Filter the DataFrame
    filtered_df = df[df['Type'] != 'Non-Penny']

    # Extract 'Symbol' column and convert it into a list
    penny_list = filtered_df['Symbol'].tolist()
    penny_df = pd.DataFrame(penny_list, columns=['Symbol'])
    penny_df = penny_df['Symbol']
    penny_dict = penny_df.to_dict()
    return penny_dict
def get_rates_df():
    df = pd.read_csv('/home/elliott/Development/files/rates_for_liq_2.csv')
    return df
def find_rate(row, rates_df):
    if row.get('Option/Equity') == 'Equity':
        return 0.00
    
    matching_rates = rates_df[
        (rates_df['Exchange'] == row['Exchange']) &
        (rates_df['Penny/Non-Penny'] == row['Penny']) &
        (rates_df['Spread/Single'] == row['Spread/Single']) &
        (rates_df['Liquidity'].astype(str) == row['liq_code']) &
        ((rates_df['Symbol'] == row['Symbol']) | (rates_df['Symbol'].isna()))
    ]
    
    if not matching_rates.empty:
        return matching_rates.iloc[0]['Rate']
    else:
        return 'Default rate'
def lookup_rate(activity_df, nested_dict, default_rate='Not Found'):
    rates = []
    print(f"Starting lookup for {len(activity_df)} rows.")  # Debugging line
    
    for index, row in activity_df.iterrows():
        trade_row = {
            'Exchange': row['Exchange'],
            'Penny': row['Penny'],
            'Complex/Simple': row['Spread/Single'],
            'Liquidity': row['liq_code'],
            'Symbol': row['Symbol']
        }
        
        print(f"Row {index + 1}: {trade_row}")  # Debugging line
        rate, _ = find_rate(trade_row, nested_dict)
        
        if rate == 'Not Found':
            print(f"Row {index + 1}: Default rate used.")  # Debugging line
            rate = default_rate
        
        rates.append(rate)
        
    print("Lookup complete.")  # Debugging line

    # Add the rates as a new column to the activity_df
    activity_df['Rate'] = rates
def convert_rate(rate):
    try:
        if isinstance(rate, str) and rate.startswith('(') and rate.endswith(')'):
            return -float(rate[1:-1])
        else:
            return float(rate)
    except ValueError:
        return None
def run(date):
    # Step 1: Get activity DataFrame
    activity_df = activity_run(date)

    rates_df = get_rates_df()
    activity_df['Rate'] = activity_df.apply(lambda row: find_rate(row, rates_df), axis=1)
    # Convert 'Quantity' and 'Rate' to float, handling the parentheses case for 'Rate'
    activity_df['Quantity'] = activity_df['Quantity'].apply(lambda x: float(x) if pd.notnull(x) else None)
    activity_df['Rate'] = activity_df['Rate'].apply(lambda x: convert_rate(x) if pd.notnull(x) else None)

    # Now compute the new columns
    activity_df['exchange_total'] = activity_df.apply(lambda row: row['Rate'] * abs(row['Quantity']) if pd.notnull(row['Rate']) and pd.notnull(row['Quantity']) else None, axis=1)
    #activity_df['comm/fee'] = activity_df['Quantity'].apply(lambda x: abs(x) * 0.26 if pd.notnull(x) else None)
    #activity_df['total_fee'] = activity_df.apply(lambda row: row['exchange_total'] + row['comm/fee'] if pd.notnull(row['exchange_total']) and pd.notnull(row['comm/fee']) else None, axis=1)
    activity_df.rename(columns={'Spread/Single': 'Complex/Simple'}, inplace=True)
    activity_df['Complex/Simple'] = activity_df['Complex/Simple'].apply(lambda x: 'Complex' if x == 'Spread' else 'Simple')
    activity_df.loc[activity_df['Symbol'] == 'NONE', 'Symbol'] = activity_df['Contract/Symbol']
    activity_df['Quantity'] = activity_df['Quantity'].abs()
    # Get the list of all columns
    all_columns = activity_df.columns.tolist()

    # Columns to be moved to the end
    cols_to_move = ['Account Num', 'Quantity', 'Symbol', 'liq_code', 'Penny', 'Complex/Simple', 'Rate']

    # Remaining columns
    remaining_cols = [col for col in all_columns if col not in cols_to_move]

    # Reorder columns
    new_order = remaining_cols + cols_to_move

    # Create new DataFrame with columns in new_order
    activity_df = activity_df[new_order]
    activity_df.to_csv(f"Scalp_daily_activity_{date}_report.csv", index=False)
    
date = '2023-09-15'
result = run(date)

/tmp/ipykernel_1214264/3587020375.py:99: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df[["Symbol", "Expiration", "Call/Put", "Strike"]] = df["Contract/Symbol"].apply(lambda x: extract_data(x))


Debug: Exchange in ['XCBO', 'EDGO', 'C2OX', 'BATO'], but TAG7000 is empty. Row: Date & Time        2023-09-15 09:50:43.850000-04:00
OrderID                             320040202502070
Account Num                                3OB05283
Contract/Symbol                 MARA20260116C17.000
Quantity                              -1.0000000000
Price                                  4.1000000000
ExecID                                 721735738932
trading_date                             2023-09-15
TAG9882                                          RN
TAG7000                                            
Exchange                                       EDGO
Broker                                         Dash
is_spread                                     False
contra_broker                                      
contra_trader                                      
contra_capacity                                    
contra_cmta                                        
Symbol                              